In [1]:
# ft articles data [heading, link, date]

import requests
from bs4 import BeautifulSoup

news_data = []
keyword = 'exxon'
            

for page in range(1, 2):
    url = f"https://www.ft.com/search?q={keyword}&page={page}&sort=date&isFirstView=true"
    result = requests.get(url)
    result_content = result.content
    soup = BeautifulSoup(result_content, "lxml")
    
    for article in soup.findAll("div", {"class": "o-teaser"}):
        heading = article.find("div", {"class": "o-teaser__heading"}).get_text(strip=True) #find(text=True), find text is outdated
        link = article.find("div", {"class": "o-teaser__heading"}).find('a', href=True)
        date = article.find('time', {"class": "o-teaser__timestamp-date"})

        if heading and link and date:
            news_data.append([heading, link['href'], date.text.strip()])

for item in news_data:
    print(f"Heading: {item[0]}, Link: {item[1]}, Date: {item[2]}")

print(f"Total articles collected: {len(news_data)}")

Heading: Oil industry eyes petrochemicals profits amid uncertain fossil fuel demandPremiumcontent, Link: /content/d1e582ea-4b08-418b-aa33-18dc943688c1, Date: July 8, 2024
Heading: Proxy season results show support for ESG efforts continues to ebb, Link: /content/1089ff59-47b9-41a2-83b1-98a205587b23, Date: July 5, 2024
Heading: Letter: Here’s my simple question about Guyana’s oil bonanza, Link: /content/c4f06b34-39f4-45d6-bda4-2c089cb44020, Date: July 1, 2024
Heading: Keep faith in China, Li Qiang tells business at ‘Summer Davos’, Link: /content/1f04f30c-a8c1-4b2e-83cc-dd0a4a8ee350, Date: June 28, 2024
Heading: Trump would be ‘devastating’ for offshore wind, warns Massachusetts governorPremiumcontent, Link: /content/0190f812-f6a0-4662-b8f8-a0c8077a70b0, Date: June 27, 2024
Heading: Business bridles at cost of green transition, Link: /content/f41bd09a-a870-49fa-8a85-6138c6469d56, Date: June 26, 2024
Heading: Transcript: WillExxonmake or break Guyana?, Link: /content/a96ae9c3-1013-4dbb-b2

In [7]:
# set up langchain env 

import os
from dotenv import load_dotenv

load_dotenv()

LANGCHAIN_TRACING_V2 = os.getenv("LANGCHAIN_TRACING_V2")
LANGCHAIN_API_KEY = os.getenv("LANGCHAIN_API_KEY")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")


In [12]:
myurl = 'https://www.ft.com' + news_data[1][1]
print(myurl)

https://www.ft.com/content/1089ff59-47b9-41a2-83b1-98a205587b23


In [13]:
import bs4
from langchain_community.document_loaders import WebBaseLoader

myurl = 'https://www.ft.com' + news_data[0][1]

bs4_strainer = bs4.SoupStrainer(class_=("o-topper__headline o-topper__headline--large", "n-content-body js-article__content-body"))
loader = WebBaseLoader(
    web_paths=(myurl,),
    bs_kwargs={"parse_only": bs4_strainer},
)
docs = loader.load()

len(docs[0].page_content)

7134

In [14]:
print(docs[0].page_content[:5000])

This article is an onsite version of our Moral Money newsletter. Premium subscribers can sign up here to get the newsletter delivered three times a week. Standard subscribers can upgrade to Premium here, or explore all FT newsletters.Visit our Moral Money hub for all the latest ESG news, opinion and analysis from around the FT Welcome back. For today’s newsletter, I look at an area of growing importance for global oil companies anxious about slowing demand for hydrocarbon fuels.Petrochemicals, which go into plastics, polyester and many other cheap and lightweight commodities that underpin modern life, could buoy the profits of oil companies even after demand for transport fuels has peaked. I look at how the fight for market share in the petrochemicals business is heating up — as well as the threat it might face from green alternatives.Thanks for reading.PetrochemicalsPetrochemicals promise a future to oil companiesPetrochemicals hold the hope of the oil industry. With the US and Europe

In [16]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=200, add_start_index=True
)
all_splits = text_splitter.split_documents(docs)

len(all_splits)

9

In [17]:
len(all_splits[0].page_content)

1000

In [20]:
# all_splits[9].metadata

In [22]:
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings

vectorstore = Chroma.from_documents(documents=all_splits, embedding=OpenAIEmbeddings())

In [27]:
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 6})

retrieved_docs = retriever.invoke("What was said about Exxon?")

len(retrieved_docs)
print(retrieved_docs[0].page_content)

as evidence of Gulf states’ long-term strategy.US oil majors have also been part of the action. Exxon is building its own petrochemical complex in southern China’s Guangdong province, as well as expanding its own chemical production at existing facilities on the US Gulf Coast.The drive for greener optionsAmid tough competition for the fastest-growing segment of global oil demand, some are betting that there is scope for further disruption — replacing petrochemicals with greener alternatives.At a refinery south-east of Paris, which was originally configured to take in crude oil, TotalEnergies is building a recycling facility. UK petrochemicals group Ineos, run by billionaire Sir Jim Ratcliffe, is developing a new ethylene cracker in the port of Antwerp that it claims would be the greenest in Europe, representing “the largest investment in the European chemical sector for a generation”.Meanwhile, in the US a crowd of start-ups have emerged that are seeking to convert biomass such as


In [28]:
from langchain_openai import ChatOpenAI
import getpass
import os
import langchainhub
from langchain import hub

llm = ChatOpenAI(model="gpt-3.5-turbo-0125")

In [29]:
prompt = hub.pull("rlm/rag-prompt")
print(prompt)

example_messages = prompt.invoke(
    {"context": "filler context", "question": "filler question"}
).to_messages()

example_messages
print(example_messages[0].content)

input_variables=['context', 'question'] metadata={'lc_hub_owner': 'rlm', 'lc_hub_repo': 'rag-prompt', 'lc_hub_commit_hash': '50442af133e61576e74536c6556cefe1fac147cad032f4377b60c436e6cdcb6e'} messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"))]
You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.
Question: filler question 
Context: filler context 
Answer:


In [30]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough


def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

for chunk in rag_chain.stream("What was said about Exxon?"):
    print(chunk, end="", flush=True)

Exxon is building its own petrochemical complex in southern China’s Guangdong province and expanding its chemical production at existing facilities on the US Gulf Coast. The fight for market share in the petrochemicals business is heating up, with the explosion of demand for petrochemicals being overwhelmingly a Chinese story. US oil majors, including Exxon, are part of the action in the petrochemical sector.